# Titanic
 
#### Data Dictionary

Variable  |	Definition   
----------|-----------  
survival  |	Survival	 0 = No, 1 = Yes
pclass	| Ticket class	1 = 1st, 2 = 2nd, 3 = 3rd
sex	| Sex	
Age	| Age in years	
sibsp	|of siblings / spouses aboard the Titanic	
parch	|of parents / children aboard the Titanic	
ticket	|Ticket number	
fare	|Passenger fare	
cabin	|Cabin number	
embarked|	Port of Embarkation	C = Cherbourg, Q = Queenstown, S = Southampton

- Variable Notes

    - pclass: A proxy for socio-economic status (SES)  
        1st = Upper  
        2nd = Middle  
        3rd = Lower  

    - age: Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5

    - sibsp: The dataset defines family relations in this way...
    - Sibling = brother, sister, stepbrother, stepsister
    - Spouse = husband, wife (mistresses and fiancés were ignored)

    - parch: The dataset defines family relations in this way...
    - Parent = mother, father
    - Child = daughter, son, stepdaughter, stepson
    - Some children travelled only with a nanny, therefore parch=0 for them.

### Imports

In [352]:
# Visualização
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 

# Analise e Manipulação
import pandas as pd
import numpy as np

# Aprendizado de maquina
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

### Lendo dados

In [353]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
combine = [train_df, test_df]

In [354]:
print (train_df.columns.values)

['PassengerId' 'Survived' 'Pclass' 'Name' 'Sex' 'Age' 'SibSp' 'Parch'
 'Ticket' 'Fare' 'Cabin' 'Embarked']


In [355]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [356]:
train_df.info()
print ('_' * 40)
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB
________________________________________
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null

In [357]:
train_df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


### Analisando dados

In [358]:
train_df[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,Pclass,Survived
0,1,0.629630
1,2,0.472826
2,3,0.242363


In [359]:
train_df[["Sex", "Survived"]].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,Sex,Survived
0,female,0.742038
1,male,0.188908


In [360]:
train_df[["SibSp", "Survived"]].groupby(['SibSp'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,SibSp,Survived
1,1,0.535885
2,2,0.464286
0,0,0.345395
3,3,0.250000
4,4,0.166667
5,5,0.000000
6,8,0.000000


### Removendo colunas

In [361]:
print("Before", train_df.shape, test_df.shape, combine[0].shape, combine[1].shape)

train_df = train_df.drop(['Ticket', 'Cabin', 'Name'], axis=1)
test_df = test_df.drop(['Ticket', 'Cabin', 'Name'], axis=1)
combine = [train_df, test_df]

print ("After", train_df.shape, test_df.shape, combine[0].shape, combine[1].shape)

Before (891, 12) (418, 11) (891, 12) (418, 11)
After (891, 9) (418, 8) (891, 9) (418, 8)


In [362]:
train_df[["Parch", "Survived"]].groupby(['Parch'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,Parch,Survived
3,3,0.600000
1,1,0.550847
2,2,0.500000
0,0,0.343658
5,5,0.200000
4,4,0.000000
6,6,0.000000


In [363]:
train_df = train_df.drop(['PassengerId'], axis=1)
combine = [train_df, test_df]
train_df.shape, test_df.shape

((891, 8), (418, 8))

### Discretizando dados

#### Sexo

In [364]:
sexos = {'female': 1, 'male': 0}
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map(sexos).astype(int)

In [365]:
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,22.0,1,0,7.2500,S
1,1,1,1,38.0,1,0,71.2833,C
2,1,3,1,26.0,0,0,7.9250,S
3,1,1,1,35.0,1,0,53.1000,S
4,0,3,0,35.0,0,0,8.0500,S


#### Fare

In [366]:
for dataset in combine:
    dataset.Fare = dataset.Fare.map(lambda x: np.nan if x == 0 else x)

In [367]:
for dataset in combine:
    classmeans = pd.pivot_table(dataset, 'Fare', 'Pclass', aggfunc='mean')
    dataset.Fare = dataset[['Fare', 'Pclass']].apply(lambda x: classmeans[x['Pclass']] if pd.isnull(x['Fare']) else x['Fare'], axis=1)

#### Age

In [368]:
for dataset in combine:
    media = np.mean(dataset.Age)
    dataset.Age = dataset.Age.fillna(media)

#### Embarcados

In [369]:
for dataset in combine:
    moda = dataset['Embarked'].mode()[0]
    dataset.Embarked.fillna(moda, inplace=True)

In [370]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null int64
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Embarked    891 non-null object
dtypes: float64(2), int64(5), object(1)
memory usage: 55.8+ KB


In [371]:
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,22.0,1,0,7.2500,S
1,1,1,1,38.0,1,0,71.2833,C
2,1,3,1,26.0,0,0,7.9250,S
3,1,1,1,35.0,1,0,53.1000,S
4,0,3,0,35.0,0,0,8.0500,S


In [372]:
embarque = {'S': 0, 'C': 1, 'Q': 2}
for dataset in combine:
    dataset.Embarked = dataset.Embarked.map(embarque).astype(int)

In [373]:
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,22.0,1,0,7.2500,0
1,1,1,1,38.0,1,0,71.2833,1
2,1,3,1,26.0,0,0,7.9250,0
3,1,1,1,35.0,1,0,53.1000,0
4,0,3,0,35.0,0,0,8.0500,0


In [374]:
train_df['AgeBand'] = pd.cut(train_df['Age'], 5)
train_df[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

,AgeBand,Survived
0,"(0.34, 16.336]",0.550000
1,"(16.336, 32.252]",0.344168
2,"(32.252, 48.168]",0.404255
3,"(48.168, 64.084]",0.434783
4,"(64.084, 80]",0.090909


#### Separando Age em 5 intervalos

In [375]:
for dataset in combine:    
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age'].astype(int)
    
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,AgeBand
0,0,3,0,1.0,1,0,7.2500,0,"(16.336, 32.252]"
1,1,1,1,2.0,1,0,71.2833,1,"(32.252, 48.168]"
2,1,3,1,1.0,0,0,7.9250,0,"(16.336, 32.252]"
3,1,1,1,2.0,1,0,53.1000,0,"(32.252, 48.168]"
4,0,3,0,2.0,0,0,8.0500,0,"(32.252, 48.168]"


In [376]:
train_df = train_df.drop(['AgeBand'], axis=1)
combine = [train_df, test_df]

#### Separando Fare em  4 intervalos

In [377]:
train_df['FareBand'] = pd.qcut(train_df['Fare'], 4)
train_df[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean().sort_values(by='FareBand', ascending=True)

,FareBand,Survived
0,"[4.0125, 7.925]",0.225664
1,"(7.925, 14.5]",0.286364
2,"(14.5, 31.275]",0.440000
3,"(31.275, 512.329]",0.586364


In [378]:
for dataset in combine:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

train_df = train_df.drop(['FareBand'], axis=1)
combine = [train_df, test_df]

In [379]:
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,1.0,1,0,0,0
1,1,1,1,2.0,1,0,3,1
2,1,3,1,1.0,0,0,1,0
3,1,1,1,2.0,1,0,3,0
4,0,3,0,2.0,0,0,1,0


In [380]:
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,1.0,1,0,0,0
1,1,1,1,2.0,1,0,3,1
2,1,3,1,1.0,0,0,1,0
3,1,1,1,2.0,1,0,3,0
4,0,3,0,2.0,0,0,1,0


### Pre Processamento

In [381]:
X_train = train_df.drop('Survived', axis=1)
y_train = train_df.Survived
X_test = test_df.drop("PassengerId", axis=1).copy()
X_train.shape, y_train.shape, X_test.shape

((891, 7), (891,), (418, 7))

In [407]:
scaler = StandardScaler().fit(X_train)
X_train_transf = scaler.transform(X_train)
X_test_transf

array([[ 0.87348191, -0.75592895,  0.07599953, ..., -0.4002477 ,
        -1.31154783,  2.24316522],
       [ 0.87348191,  1.32287566,  0.07599953, ..., -0.4002477 ,
        -1.31154783, -0.6778412 ],
       [-0.31581919, -0.75592895,  0.27963931, ..., -0.4002477 ,
        -0.43857491,  2.24316522],
       ..., 
       [ 0.87348191, -0.75592895,  0.07599953, ..., -0.4002477 ,
        -1.31154783, -0.6778412 ],
       [ 0.87348191, -0.75592895, -0.12764024, ..., -0.4002477 ,
        -0.43857491, -0.6778412 ],
       [ 0.87348191, -0.75592895, -0.12764024, ...,  0.61989583,
         0.43439801,  0.78266201]])

In [383]:
scaler_test = StandardScaler().fit(X_test)
X_test_transf = scaler_test.transform(X_test)

In [384]:
scores = {}

### Decision  Tree Classifier

In [385]:
tree = DecisionTreeClassifier()

In [386]:
tree.fit(X_train_transf, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [387]:
Y_pred = tree.predict(X_test_transf)

In [388]:
acc_decision_tree = round(tree.score(X_train_transf, y_train) * 100, 2)
acc_decision_tree

88.659999999999997

In [389]:
tree_submission = pd.DataFrame({
    'PassengerId': test_df['PassengerId'],
    'Survived': Y_pred
})

In [390]:
tree_submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,0


In [391]:
tree_submission.to_csv('submission.csv', index=False)
scores['tree'] = tree_submission

### Random Forest Classifier

In [392]:
rand_forest = RandomForestClassifier(n_estimators=100)

In [393]:
rand_forest.fit(X_train_transf, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [394]:
y_pred = rand_forest.predict(X_test_transf)

In [395]:
rand_score = rand_forest.score(X_train_transf, y_train)
acc_rand_forest = round(rand_score * 100, 2)
acc_rand_forest

88.659999999999997

In [396]:
rand_submission = pd.DataFrame({
    'PassengerId': test_df['PassengerId'],
    'Survived': y_pred
})

rand_submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,0


In [397]:
rand_submission.to_csv('submission_rand_forest.csv', index=False)
scores['rand_forest'] = rand_submission

### Kneighboors Classifier

In [398]:
knn = KNeighborsClassifier(n_neighbors=50)

In [399]:
knn.fit(X_train_transf, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=50, p=2,
           weights='uniform')

In [400]:
y_pred = knn.predict(X_test_transf)

In [401]:
knn_score = knn.score(X_train_transf, y_train)
acc_knn = round(rand_score * 100, 2)
acc_knn

88.659999999999997

In [402]:
knn_submission = pd.DataFrame({
    'PassengerId': test_df['PassengerId'],
    'Survived': y_pred
})

knn_submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [403]:
knn_submission.to_csv('submission_knn.csv', index=False)
scores['knn'] = knn_submission

### Comparacao

In [404]:
other = pd.read_csv('gender_submission.csv')
other.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [405]:
for key in scores:
    print (key, ' = ', round (accuracy_score(other.Survived, scores[key].Survived) * 100, 2))

tree  =  84.93
rand_forest  =  84.21
knn  =  97.13
